In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}


genre_codes = {
    '발라드': 'GN0100',
    '댄스': 'GN0200',
    '힙합': 'GN0300',
    '알앤비': 'GN0400',
    '인디': 'GN0500',
    '록' : 'GN0600',
    '트로트' : 'GN0700'
}

with open('melon_genre_album', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["Genre", "Rank", "Title", "Artist", "Album", "Album Description"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for genre, code in genre_codes.items():
        melon_url = f"https://www.melon.com/genre/song_list.htm?gnrCode={code}&dtlGnrCode=&orderBy=POP&steadyYn=N&colOrder=1&classCd=DP0100"

        # 인기차트 페이지에 접근
        raw = requests.get(melon_url, headers=header)
        soup = BeautifulSoup(raw.text, "html.parser")

        # 차트 목록 가져옴
        box = soup.find("tbody")

        # 각 노래의 제목, 가수, 앨범을 가져옴
        all_singer = box.find_all("div", {"class":"ellipsis rank02"})
        all_title = box.find_all("div", {"class":"ellipsis rank01"})
        all_album = box.find_all("div", {"class":"ellipsis rank03"})

        # 노래 정보를 csv에 저장
        count = 0
        for singer, title, album in zip(all_singer, all_title, all_album):
            count += 1
            album_link = album.find('a')['href']
            album_id = album_link.split("'")[1]  # 앨범 ID 추출
            album_url = f"https://www.melon.com/album/detail.htm?albumId={album_id}"

            # 앨범 페이지에 접근하여 앨범 설명 가져오기
            album_raw = requests.get(album_url, headers=header)
            album_soup = BeautifulSoup(album_raw.text, "html.parser")
            album_desc_div = album_soup.find("div", {"class": "cont_albuminfo"})
            album_desc = album_desc_div.text.strip() if album_desc_div else "No description"

            writer.writerow({
                'Genre': genre,
                'Rank': count,
                'Title': title.find('a').text.strip(),
                'Artist': singer.find('a').text.strip(),
                'Album': album.find('a').text.strip(),
                'Album Description': album_desc
            })

print("csv 파일이 성공적으로 저장됨")

csv 파일이 성공적으로 저장됨
